# Transfer Learning - Fine Tuning

In [1]:
# Importing TF and checking the version
import tensorflow as tf

print(tf.__version__)

2.10.1


In [2]:
# Importing Daniel Bourke's helper functions
from DanielBourke_HelperFunctions import create_tensorboard_callback, plot_loss_curves, walk_through_dir

In [3]:
# Check how many images are in target folders
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [4]:
# Creating training and test dir paths
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [5]:
# Creating image dataset
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_data_10percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir,
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE
)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [6]:
# Check train dataset
train_data_10percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [7]:
# Check class names
train_data_10percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [8]:
# Getting sample batch data
for images, labels in train_data_10percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[1.00948975e+02 9.92346878e+01 1.15591835e+02]
   [8.73928528e+01 9.11734695e+01 1.03270409e+02]
   [8.53418350e+01 9.27704086e+01 1.01117340e+02]
   ...
   [1.67234772e+02 1.14163452e+02 1.35520660e+02]
   [1.73163315e+02 1.32382751e+02 1.60000137e+02]
   [1.84969559e+02 1.50326736e+02 1.83097153e+02]]

  [[1.02020416e+02 1.02811226e+02 1.17020416e+02]
   [7.78367310e+01 8.18265305e+01 9.29693832e+01]
   [8.72244873e+01 9.62244873e+01 1.01367348e+02]
   ...
   [1.62668411e+02 1.12183853e+02 1.33995102e+02]
   [1.70632736e+02 1.29918503e+02 1.60132828e+02]
   [1.85688950e+02 1.50403305e+02 1.86709457e+02]]

  [[1.26877556e+02 1.28862244e+02 1.39867340e+02]
   [1.40607147e+02 1.45464294e+02 1.51020401e+02]
   [1.43413269e+02 1.51153061e+02 1.51698975e+02]
   ...
   [1.65500137e+02 1.16046173e+02 1.41954407e+02]
   [1.81142929e+02 1.41459305e+02 1.75785858e+02]
   [1.90295929e+02 1.56857254e+02 1.96515396e+02]]

  ...

  [[2.15729675e+02 1.89229660e+02 1.66801056e+02]
   [2

## Creating Transfer Learning models with Keras' Functional API

### Model 0

In [10]:
# Creating baseline model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# Freezing the model to avoid changes to pre-trained patterns
base_model.trainable = False

# Creating inputs into the model
inputs = tf.keras.layers.Input(
    shape=(224, 224, 3),
    name="input_layer"
)

# If using ResNet50V2 (not EfficientNetB0), then input needs to be re-scaled
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)(inputs)

# Passing the inputs to baseline model
x = base_model(inputs)
print(f"Shape after passing the inputs: {x.shape}.")

# Average pooling the outputs
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"Shape after GlobalPooling2D: {x.shape}.")

# Creating outputs of the model
outputs = tf.keras.layers.Dense(
    10,
    activation="softmax",
    name="output_layer"
)(x)

# Combining inputs and outputs to set a model
model_0 = tf.keras.Model(inputs, outputs)

Shape after passing the inputs: (None, 7, 7, 1280).
Shape after GlobalPooling2D: (None, 1280).
